<a href="https://colab.research.google.com/github/MudSnail/Land_Cover_Classification/blob/main/EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#import base libraries
import pandas as pd
import numpy as np
import copy
import pickle
import cloudpickle

#Sklearn - Pipelines, Training
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_selection import SelectKBest
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler, MaxAbsScaler, RobustScaler

#Classification Models
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

#Model metrics
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, f1_score, roc_auc_score, precision_score, recall_score

#Visualizations
import matplotlib.pyplot as plt
import seaborn as sns

#misc
import warnings
warnings.filterwarnings('ignore')

In [2]:
#Connect to drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Input Data, Combine and Subsample

In [3]:
#Read in CSVs
james_bay = pd.read_csv('/content/drive/MyDrive/james_bay_data.csv')
labrador = pd.read_csv('/content/drive/MyDrive/labrador_data.csv')
simcoe = pd.read_csv('/content/drive/MyDrive/simcoe_york_data.csv')

In [4]:
#List dataframes to add to james_bay
dfs = [labrador, simcoe]

#Merged data
merged_data = james_bay.append(dfs)

#Print shape and check new data
print(merged_data.shape)
merged_data.head()

(8917500, 13)


,B01,B06,B11,B07,B05,B03,B09,B12,B8A,B04,B08,B02,Class
0,0.0055,0.0017,0.0009,0.0016,0.0026,0.0056,0.0001,0.0009,0.0003,0.0032,0.0012,0.0114,0
1,0.0054,0.0020,0.0009,0.0017,0.0021,0.0060,0.0001,0.0005,0.0007,0.0028,0.0014,0.0115,0
2,0.0058,0.0018,0.0007,0.0016,0.0028,0.0060,0.0001,0.0008,0.0007,0.0030,0.0012,0.0124,0
3,0.0057,0.0016,0.0007,0.0016,0.0023,0.0058,0.0001,0.0012,0.0006,0.0031,0.0011,0.0119,0
4,0.0066,0.0013,0.0008,0.0020,0.0029,0.0059,0.0001,0.0008,0.0005,0.0030,0.0014,0.0115,0


In [5]:
#Drop Class = 0 (null values, which equate to 18)
merged_data = merged_data[merged_data.Class != 0]

#Double check value counts
merged_data.Class.value_counts()

15    2836714
18    1050988
1      677877
5      666952
16     646870
13     612009
8      575175
17     512930
12     415493
6      378274
2      151560
11     135890
14      94850
10      86338
19      74619
Name: Class, dtype: int64

In [6]:
#SubSample ~70 000 for each class
sample = merged_data.groupby('Class').apply(lambda x: x.sample(n=70000)).reset_index(drop=True)
sample.head(4)

,B01,B06,B11,B07,B05,B03,B09,B12,B8A,B04,B08,B02,Class
0,0.0062,0.1527,0.1110,0.1718,0.0765,0.0406,0.1873,0.0585,0.1932,0.0282,0.1896,0.0193,1
1,0.0097,0.1447,0.1135,0.1621,0.0737,0.0395,0.1792,0.0609,0.1792,0.0309,0.1759,0.0225,1
2,0.0130,0.1325,0.1062,0.1520,0.0676,0.0329,0.1689,0.0561,0.1703,0.0260,0.1642,0.0196,1
3,0.0132,0.2016,0.0895,0.2462,0.0599,0.0303,0.2585,0.0416,0.2722,0.0176,0.2557,0.0118,1


# EDA - Checking Each Layer per Class

In [7]:
def plot_class_distributions(x, y, z):
  """
  Input:
  x - the specific NRCAN Land Classification
  y - the plot's 1st x lim
  z - the plot's 2nd x lim

  Output: Skewness distribution plots per each layer for the class specified
  """
  #Select Class and drop column
  selected_class = merged_data.loc[merged_data['Class'] == x]
  selected_class = merged_data.drop('Class', axis = 1)

  #exctract column values, identify number of columns and rows
  l = selected_class.columns.values 
  number_of_columns=10
  number_of_rows = len(l)-1/number_of_columns

  #plot distribution skewness figure for each column
  plt.figure(figsize=(2*number_of_columns,5*number_of_rows))
  for i in range(0,len(l)):
    plt.subplot(number_of_rows + 1,number_of_columns,i+1)
    sns.distplot(merged_data[l[i]],kde=True)
    #Set x axis limits
    plt.xlim([y,z]) 

In [ ]:
#Check skewness of Class 1
plot_class_distributions(1, 0, 0.4)

In [ ]:
#Check skewness of Class 2
plot_class_distributions(2, 0, 0.5)

In [ ]:
#Check skewness of Class 5
plot_class_distributions(5,0, 0.6)

In [ ]:
#Check skewness of Class 6
plot_class_distributions(6,0,0.6)

In [ ]:
#Check skewness of Class 8
plot_class_distributions(8,0,0.6)

In [ ]:
#Check skewness of Class 10
plot_class_distributions(10,0,0.6)

In [ ]:
#Check skewness of Class 11
plot_class_distributions(11,0, 0.6)

In [ ]:
#Check skewness of Class 12
plot_class_distributions(12,0, 0.6)

In [ ]:
#Check skewness of Class 13
plot_class_distributions(13,0, 0.6)

In [ ]:
#Check skewness of Class 14
plot_class_distributions(14,0,0.6)

In [ ]:
#Check skewness of Class 15
plot_class_distributions(15,0, 0.6)

In [ ]:
#Check skewness of Class 16
plot_class_distributions(16,0,0.6)

In [ ]:
#Check skewness of Class 17
plot_class_distributions(17,0, 0.6)

In [ ]:
#Check skewness of Class 18
plot_class_distributions(18,0,0.3)

In [ ]:
#Check skewness of Class 19
plot_class_distributions(19,-0.2,1)

#Subsample and Save final csv

In [ ]:
#SubSample ~25 000 for each class
sample = merged_data.groupby('Class').apply(lambda x: x.sample(n=50000)).reset_index(drop=True)
sample.head(4)